In [ ]:

%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset

# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',
                 load_sensor_names=['C3', 'C4']).load()
cnt = cnt.drop_channels(['STI 014'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

log.info("Resampling train...")
cnt = resample_cnt(cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 2.0)
train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

In [ ]:
inputs = np_to_var(x_rest[:10,0:1,:,None], dtype=np.float32).squeeze()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(var_to_np(inputs[:,0]), var_to_np(inputs[:,1]), marker='o', ls='')

In [ ]:
from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20160116, False)
fakes = th.randn(200,2) + 0.5

fig = plt.figure(figsize=(5,5))
plt.plot(var_to_np(fakes.squeeze()[:,0]), var_to_np(fakes.squeeze()[:,1]), marker='o', ls='')
plt.plot(var_to_np(inputs.squeeze()[:,0]), var_to_np(inputs.squeeze()[:,1]), marker='o', ls='')
display(fig)
plt.close(fig)
adv_model = nn.Sequential(
    ResidualBlock(
        nn.Sequential(
            nn.Linear(2,100),
            nn.ReLU(),
            nn.Linear(100,2)),
    ),
    ResidualBlock(
        nn.Sequential(
            nn.Linear(2,100),
            nn.ReLU(),
            nn.Linear(100,2)),
    ),
    nn.Linear(2,1)
    )

from spectral_norm import spectral_norm

for module in adv_model.modules():
    if hasattr(module, 'weight'):
        spectral_norm(module, n_power_iterations=3, to_norm=0.9)

# warm start spectral norm
for _ in range(100):
    adv_model(inputs)
optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-2, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))

score_fake = adv_model(fakes).squeeze()

fig = plt.figure(figsize=(5,5))
max_abs_score = th.max(th.abs(score_fake)).item()
im = plt.scatter(var_to_np(fakes[:,0]), var_to_np(fakes[:,1]),cmap=cm.coolwarm,
       c=var_to_np(score_fake), vmin=-max_abs_score, vmax=max_abs_score)
plt.plot(var_to_np(inputs[:,0]), var_to_np(inputs[:,1]),
         marker='o', ls='', color=seaborn.color_palette()[2])
plt.legend(('Real data',))
cbar = plt.colorbar(im)
cbar.set_label('Critic Score')
plt.title("Generated points and critic score")
display(fig)
plt.close(fig)


In [ ]:
in_np = var_to_np(inputs).squeeze()
fake_np = var_to_np(fakes).squeeze()
import ot



dist = np.sum(np.abs(in_np[:,None] - fake_np[None]), axis=2)

match_matrix = ot.emd([],[], dist)

cost = np.sum(dist * match_matrix)

cost

In [ ]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
n_epochs = 2001
for i_epoch in range(n_epochs):
    score_fake = adv_model(fakes)
    score_real = adv_model(inputs)
    d_loss = nn.functional.relu(1.0 - score_real).mean() + nn.functional.relu(
                1.0 + score_fake).mean()
    optim_adv.zero_grad()
    d_loss.backward()
    optim_adv.step()
    d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
    wd_dist = th.mean(score_real) - th.mean(score_fake)
    epoch_row = {
    'd_loss': d_loss.item(),
    'o_real': th.mean(score_real).item(),
    'o_fake': th.mean(score_fake).item(),
    'd_grad': d_grad,
    'wd_diff': cost - wd_dist.item()}
    df = df.append(epoch_row, ignore_index=True)
    

    if i_epoch % (n_epochs // 20) == 0:
        display_text("Epoch {:d}".format(i_epoch))
        display(df.iloc[-5:])
        score_fake = adv_model(fakes).squeeze()
        fig = plt.figure(figsize=(5,5))
        max_abs_score = th.max(th.abs(score_fake)).item()
        im = plt.scatter(var_to_np(fakes[:,0]), var_to_np(fakes[:,1]),cmap=cm.coolwarm,
               c=var_to_np(score_fake), vmin=-max_abs_score, vmax=max_abs_score)
        plt.plot(var_to_np(inputs[:,0]), var_to_np(inputs[:,1]),
                 marker='o', ls='', color=seaborn.color_palette()[2])
        plt.legend(('Real data',))
        cbar = plt.colorbar(im)
        cbar.set_label('Critic Score')
        plt.title("Generated points and critic score")
        display(fig)
        plt.close(fig)

### let's try larger standard deviation

In [ ]:
from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(20160116, False)
fakes = th.randn(200,2) * 5 + 0.5

fig = plt.figure(figsize=(5,5))
plt.plot(var_to_np(fakes.squeeze()[:,0]), var_to_np(fakes.squeeze()[:,1]), marker='o', ls='')
plt.plot(var_to_np(inputs.squeeze()[:,0]), var_to_np(inputs.squeeze()[:,1]), marker='o', ls='')
display(fig)
plt.close(fig)
adv_model = nn.Sequential(
    ResidualBlock(
        nn.Sequential(
            nn.Linear(2,200),
            nn.ReLU(),
            nn.Linear(200,2)),
    ),
    ResidualBlock(
        nn.Sequential(
            nn.Linear(2,200),
            nn.ReLU(),
            nn.Linear(200,2)),
    ),
    nn.Linear(2,1)
    )

adv_model = nn.Sequential(
    nn.Linear(2,100),
    nn.ReLU(),
    nn.Linear(100,100),
    nn.ReLU(),
    nn.Linear(100,10),
    nn.ReLU(),
    nn.Linear(10,1)
    )

from spectral_norm import spectral_norm

for module in adv_model.modules():
    if hasattr(module, 'weight'):
        spectral_norm(module, n_power_iterations=3, to_norm=1)

# warm start spectral norm
for _ in range(100):
    adv_model(inputs)
optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-2, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))

score_fake = adv_model(fakes).squeeze()

fig = plt.figure(figsize=(5,5))
max_abs_score = th.max(th.abs(score_fake)).item()
im = plt.scatter(var_to_np(fakes[:,0]), var_to_np(fakes[:,1]),cmap=cm.coolwarm,
       c=var_to_np(score_fake), vmin=-max_abs_score, vmax=max_abs_score)
plt.plot(var_to_np(inputs[:,0]), var_to_np(inputs[:,1]),
         marker='o', ls='', color=seaborn.color_palette()[2])
plt.legend(('Real data',))
cbar = plt.colorbar(im)
cbar.set_label('Critic Score')
plt.title("Generated points and critic score")
display(fig)
plt.close(fig)


In [ ]:
in_np = var_to_np(inputs).squeeze()
fake_np = var_to_np(fakes).squeeze()
import ot



dist = np.sum(np.abs(in_np[:,None] - fake_np[None]), axis=2)

match_matrix = ot.emd([],[], dist)

cost = np.sum(dist * match_matrix)

cost

In [ ]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
n_epochs = 2001
for i_epoch in range(n_epochs):
    score_fake = adv_model(fakes)
    score_real = adv_model(inputs)
    #d_loss = nn.functional.relu(1.0 - score_real).mean() + nn.functional.relu(
    #            1.0 + score_fake).mean()
    d_loss = -score_real.mean() + score_fake.mean()
    optim_adv.zero_grad()
    d_loss.backward()
    optim_adv.step()
    d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
    wd_dist = th.mean(score_real) - th.mean(score_fake)
    epoch_row = {
    'd_loss': d_loss.item(),
    'o_real': th.mean(score_real).item(),
    'o_fake': th.mean(score_fake).item(),
    'd_grad': d_grad,
    'wd_diff': cost - wd_dist.item()}
    df = df.append(epoch_row, ignore_index=True)
    

    if i_epoch % (n_epochs // 20) == 0:
        display_text("Epoch {:d}".format(i_epoch))
        display(df.iloc[-5:])
        score_fake = adv_model(fakes).squeeze()
        fig = plt.figure(figsize=(5,5))
        max_abs_score = th.max(th.abs(score_fake)).item()
        im = plt.scatter(var_to_np(fakes[:,0]), var_to_np(fakes[:,1]),cmap=cm.coolwarm,
               c=var_to_np(score_fake), vmin=-max_abs_score, vmax=max_abs_score)
        plt.plot(var_to_np(inputs[:,0]), var_to_np(inputs[:,1]),
                 marker='o', ls='', color=seaborn.color_palette()[2])
        plt.legend(('Real data',))
        cbar = plt.colorbar(im)
        cbar.set_label('Critic Score')
        plt.title("Generated points and critic score")
        display(fig)
        plt.close(fig)

In [ ]:
cost - (th.mean(score_real) - th.mean(score_fake))

In [ ]:
from torch import autograd
test_fakes = (th.randn(200,2) * 5 + 0.5).requires_grad_(True)
disc_interpolates = adv_model(test_fakes)
gradients = autograd.grad(outputs=disc_interpolates, inputs=test_fakes,
                          grad_outputs=torch.ones(disc_interpolates.size()),
create_graph=True, retain_graph=False, only_inputs=True)[0]

grad_norms = gradients.norm(2, dim=1)
plt.figure(figsize=(8,2))
seaborn.distplot(var_to_np(grad_norms))
plt.plot(var_to_np(grad_norms), var_to_np(th.randn(len(gradients)) * 0.1), 
         ls='', marker='o',
        alpha=0.5)
plt.ylim(-0.4,5)
plt.xlabel('Gradient L2-Norm')


In [ ]:
for _ in range(100):
    score_fake = adv_model(fakes)
    score_real = adv_model(inputs)
    print(cost - (th.mean(score_real) - th.mean(score_fake)))

In [ ]:
score_fake = adv_model(fakes).squeeze()
score_real = adv_model(inputs).squeeze()
score_fake = score_fake - ((score_fake.mean() + score_real.mean()) / 2)

fig = plt.figure(figsize=(5,5))
max_abs_score = th.max(th.abs(score_fake)).item()
im = plt.scatter(var_to_np(fakes[:,0]), var_to_np(fakes[:,1]),cmap=cm.coolwarm,
       c=var_to_np(score_fake), vmin=-max_abs_score, vmax=max_abs_score)
plt.plot(var_to_np(inputs[:,0]), var_to_np(inputs[:,1]),
         marker='o', ls='', color=seaborn.color_palette()[2])
plt.legend(('Real data',))
cbar = plt.colorbar(im)
cbar.set_label('Critic Score')
plt.title("Generated points and critic score")
display(fig)
plt.close(fig)